In [1]:
import pandas as pd
import numpy as np
import math
import os

%run auxiliar.ipynb

In [2]:
data = pd.read_csv("./database/LocTeste.csv")

Primeiro é necessário criar um mapa de rádio para que se possa fazer a busca, aqui iremos criar um grid de 50x50 para representar o mapa onde cada célula representa uma região. Cada célula é representada por sua coordenada geográfica, cada célula é um possível lugar para o EM, porém é necessário que cada célula tenha seu fingerprint.

In [3]:
#Latitude e longitude maximo e mínimo do db
LAT_INI  = min(data['lat'])
LAT_FIN  = max(data['lat'])
LONG_INI = min(data['lon'])
LONG_FIN = max(data['lon'])

In [4]:
#Tamanho em metros do grid
y_tam = latToKm(LAT_INI, LAT_FIN)*1000
x_tam = latToKm(LONG_INI, LONG_FIN)*1000
print(y_tam)
print(x_tam)

1855.0925599999555
2212.0889899998224


In [5]:
#Tamanho em metros da celula
cell_tam = 20

#Numero de celulas em x e em y
n_cell_x = math.ceil((x_tam / cell_tam))
n_cell_y = math.ceil((y_tam / cell_tam))

print(n_cell_x)
print(n_cell_y)

111
93


In [6]:
#Incremento na Latitude
inc_lat = (LAT_FIN - LAT_INI) / n_cell_x
inc_lng = (LONG_FIN - LONG_INI) / n_cell_y

result = []

for i in range(n_cell_x):
    for j in range(n_cell_y):
        lat = LAT_INI + i * inc_lat
        lng = LONG_INI + j * inc_lng
        result.append((lat, lng))

# salva para o CSV
fingerprint_map = pd.DataFrame(result, columns=['lat', 'lon'])
fingerprint_map.to_csv("./grid/grid_{}.csv".format(cell_tam), index=False)


Com os grids gerados, podemos começar a aplicar um modelo para gerar as fingerprints de Path Loss para cada uma das células do mapa.

Aqui podemos usar dois modelos, Okumura-Hata ou o Ericson 9999, vamos começar tentando com o Okumura-Hata, como não temos acesso a altura da base e da antena colocaremos um valor médio de 100m e 5m em cada uma.

Hata Model

In [7]:
def correction_factor(altura_antena):
    return 3.2*(np.log10(11.75*altura_antena))**2 - 4.95

def hata_model(frequency, altura_base, altura_antena, distancia):
    med_loss = (65.55 + 26.26*np.log10(frequency) - 13.82*np.log10(altura_base) \
                - correction_factor(altura_antena) + \
                (44.9 - 6.55*np.log10(altura_base))*np.log10(distancia))
    return 2*(med_loss)


Freespace Model

In [8]:
def freespace_model(frequency, altura_base, altura_antena, distancia):
    med_loss = 32.44 + 20*np.log10(distancia) + 20*np.log10(frequency)
    return 2*(med_loss)


Coste231 Model

In [9]:
def correction_factor_cost(frequency,altura_antena):
    return ((1.1*(np.log10(frequency))-0.7)*altura_antena - (1.56*(np.log10(frequency))-0.8))

def cost231_model(frequency, altura_base, altura_antena, distancia):
    med_loss = 2*(46.3 + 33.9*np.log10(frequency) - 13.82*np.log10(altura_base) \
                - correction_factor_cost(frequency,altura_antena)  + \
                (44.9 - 6.55*np.log10(altura_base))*np.log10(distancia))
    return med_loss


Ericson Model

In [10]:
def ericson_model(frequency, altura_base, altura_antena, distancia):
    g = 44.49*np.log10(frequency)-4.78*(np.log10(frequency)**2)
    a2= 12
    a3= 0.1
    a0,a1 = 43.2,68.9
    med_loss=a0+a1*np.log10(distancia)+a2*np.log10(altura_base) \
            +a3*(np.log10(altura_base))*(np.log10(distancia)) \
            -3.2*np.log10((11.75*altura_antena)**2)+g
    return (2*med_loss)

ITU-R Model

In [11]:
def itur_model(frequency, altura_base, altura_antena, distancia):
    med_loss = 2*(40*np.log10(distancia) + 30*np.log10(frequency) + 49)
    return med_loss

Aqui calculamos o pathloss e preenchemos um csv de fingerprint com os valores dos Pathloss para serem comparados futuramente

In [12]:

def calculate_pathloss(fingerprint, erb):
    """
    Calcula todos os Path Loss para todos os pontos de um fingerprint
    """
    path_list = []
    lat_erb = erb['lat']
    lon_erb = erb['lon']

    for n, point in fingerprint.iterrows():
        lat = point['lat']
        lon = point['lon']
        
        distance = two_point_distance(lat_erb, lon_erb, lat, lon)
        pathloss = cost231_model(frequency=1800, altura_base=50.0, altura_antena=1.5, distancia=distance)
        
        path_list.append(pathloss)
    
    return path_list


def generate_grid(erbs, fingerprint):
    """
    Gera um fingerprint e retorna como dataFrame
    """
    path_dict = {}
    for idx, erb in erbs.iterrows():
        BTS = erb['bts']
        path_dict[BTS] = calculate_pathloss(fingerprint, erb)
    grid = pd.DataFrame(path_dict)
    return grid


def fingerprints_to_csv():
    """
    Gera todos os fingerprints e salva em csv
    """
    cell_tam = 20
    erbs = pd.read_csv("./database/Dados_BTS.csv")
    fingerprint = pd.read_csv("./grid/grid_{}.csv".format(cell_tam))
    grid = generate_grid(erbs, fingerprint)
    grid.insert(0, 'lat', fingerprint['lat'], True)
    grid.insert(1, 'lon', fingerprint['lon'], True)
    grid.to_csv("./fingerprint/fingerprint_{}.csv".format(cell_tam), index=False)

In [13]:
fingerprints_to_csv()

Agora temos todos os fingerprints gerados